In [1]:
library(MuSiC)
library(Biobase)
library(ggplot2)
library(dplyr)
library(ggcorrplot)
library(reshape)
library(corrplot)
library(tidyr)
library(caret)

Loading required package: nnls

Loading required package: ggplot2

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.



Attaching package: ‘dplyr’


The following object is masked from ‘package:Biobase’:

    combine


The following object

In [2]:
args = commandArgs(trailingOnly=TRUE)
print("running BayesPrism with following args:")

if (length(args)==0) {
  stop("At least one argument must be supplied", call.=FALSE)
} else if (length(args)==6) {
  # default subtype location
  args[7] = args[3]
}
print(args)

[1] "running BayesPrism with following args:"
[1] "/home/cke/.local/share/jupyter/runtime/kernel-5c0b67b6-fe1e-40f9-ad68-8463d9ef97e3.json"


In [ ]:
  /home/cke/Puram/scRNAlabels/ /home/cke/Puram/markers/top20markers_de_cor_symbol.txt test_fullpipeline_1_pseudobulk_top20markers_6LOT /home/cke/PseudoBulk/Results/MuSiC/

In [35]:
args[1] <- "/home/cke/PseudoBulk/InputToWrapper/real_scRNAref.tsv"
args[2] <- "/home/cke/TCGA-HNSC.htseq_counts_exp2_symbol.tsv"
args[3] <- "/home/cke/Puram/scRNAlabels/"
args[4] <- "/home/cke/Puram/top100markers_de_cor_symbol.txt"
args[5] <- "test_fullpipeline_1_real_top20markers"
args[6] <- "/home/cke/PseudoBulk/Results/MuSiC/"
args[7] <- args[3]


In [36]:
path_scRNA <- args[1]
path_bulk <- args[2]
path_label <- args[3]
path_marker <- args[4]
FS_setup <- args[5]
path_out <- args[6]
path_label_subtype <- args[7]

In [52]:
df_Puram <- read.table(path_scRNA,sep='\t',header=TRUE,row.names = 1,check.names = FALSE)
df_Puram = t(df_Puram)

df_bulk <- read.table(path_bulk,sep='\t',header=TRUE,row.names = 1,check.names = FALSE)
df_bulk <- t(df_bulk)

label <- read.table(paste0(path_label,"cellcategory_simple.csv"),sep=',',header=TRUE,row.names = 1)
label_subtype <- read.table(paste0(path_label_subtype,"cellcategory_subtype.csv"),sep=',',header=TRUE,row.names = 1)

label$subtype <- label_subtype$cell_category
label$sampleID <- rownames(label)
label <- subset(label, row.names(label) %in% colnames(df_Puram))
label_subtype <- subset(label_subtype, row.names(label_subtype) %in% colnames(df_Puram))

if (path_marker!='noFS') {
    marker_genes <- scan(path_marker,character(),sep=',')
    }

In [38]:
# generate expressionset object and run MuSiC main function, process result cell type fraction
run_MuSiC <- function(df_Puram_filtered,df_bulk,label){
    # prepare scRNA-seq ref data, as expressionset object
    metadata <- data.frame(labelDescription= c("sampleID","cell_category", "subtype"), row.names=c("sampleID","cell_category", "subtype"))
    Puram.eset = ExpressionSet(assayData = data.matrix(df_Puram_filtered), phenoData =  new("AnnotatedDataFrame", data = label, varMetadata = metadata) )
    bulk_raw.eset = ExpressionSet(assayData = data.matrix(df_bulk))
    # Estimate cell type proportions
    Est.prop.tcga_raw = music_prop(bulk.eset = bulk_raw.eset, sc.eset = Puram.eset, clusters = 'cell_category',samples = 'sampleID')
    MuSiC_res <- data.matrix(Est.prop.tcga_raw$Est.prop.weighted)
    colnames(MuSiC_res)[5] <- 'other'
    MuSiC_res <- MuSiC_res[, order(colnames(MuSiC_res))]
    colnames(MuSiC_res)[9] <- 'T-cell'
    colnames(MuSiC_res)[1] <- 'B-cell'
    return(MuSiC_res)
}

In [45]:
metadata <- data.frame(labelDescription= c("sampleID","cell_category", "subtype"), row.names=c("sampleID","cell_category", "subtype"))
Puram.eset = ExpressionSet(assayData = data.matrix(df_Puram_filtered), phenoData =  new("AnnotatedDataFrame", data = label, varMetadata = metadata) )
bulk_raw.eset = ExpressionSet(assayData = data.matrix(df_bulk))

In [46]:
Puram.eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 915 features, 4484 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: HN28_P15_D06_S330_comb HN28_P6_G05_S173_comb ...
    HNSCC20_P3_G06_S78_comb (4484 total)
  varLabels: cell_category subtype sampleID
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

In [47]:
bulk_raw.eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 546 features, 38929 samples 
  element names: exprs 
protocolData: none
phenoData: none
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

In [39]:
label

,cell_category,subtype,sampleID
,<chr>,<chr>,<chr>
HN28_P15_D06_S330_comb,Fibroblast,Fibroblast,HN28_P15_D06_S330_comb
HN28_P6_G05_S173_comb,Fibroblast,Fibroblast,HN28_P6_G05_S173_comb
HN26_P14_D11_S239_comb,tumor,tumor 26,HN26_P14_D11_S239_comb
HN26_P14_H05_S281_comb,Fibroblast,Fibroblast,HN26_P14_H05_S281_comb
HN26_P25_H09_S189_comb,tumor,tumor 26,HN26_P25_H09_S189_comb
HN26_P14_H06_S282_comb,tumor,tumor 26,HN26_P14_H06_S282_comb
HN25_P25_C04_S316_comb,tumor,tumor 25,HN25_P25_C04_S316_comb
HN26_P25_A11_S107_comb,Fibroblast,Fibroblast,HN26_P25_A11_S107_comb
HN26_P25_C09_S129_comb,tumor,tumor 26,HN26_P25_C09_S129_comb


In [49]:
df_Puram_filtered

,HN28_P15_D06_S330_comb,HN28_P6_G05_S173_comb,HN26_P14_D11_S239_comb,HN26_P14_H05_S281_comb,HN26_P25_H09_S189_comb,HN26_P14_H06_S282_comb,HN25_P25_C04_S316_comb,HN26_P25_A11_S107_comb,HN26_P25_C09_S129_comb,HNSCC26_P24_H05_S377_comb,⋯,HNSCC20_P3_B10_S22_comb,HNSCC20_P13_B11_S215_comb,HNSCC20_P3_C08_S32_comb,HNSCC17_P4_H03_S183_comb,HNSCC20_P3_F09_S69_comb,HNSCC17_P4_G12_S180_comb,HNSCC20_P13_C05_S221_comb,HNSCC17_P4_C12_S132_comb,HNSCC20_P3_H08_S92_comb,HNSCC20_P3_G06_S78_comb
LMAN1,1.249076e+02,0.0000000,59.9100570,0.000000e+00,73.2182540,6.323838e+01,0.000000,35.1992870,0.0000000,0.7199636,⋯,0.0000000,24.880000,0.000000,0.000000e+00,5.030052e+00,1.036874e+02,0.0000000,95.919770,5.8399744,7.0400553
TAOK3,0.000000e+00,0.0000000,63.2383840,0.000000e+00,112.6691060,1.804027e+01,15.530114,108.8195300,0.0000000,0.0000000,⋯,0.0000000,24.480960,0.000000,0.000000e+00,1.519994e+00,0.000000e+00,0.0000000,37.899147,6.4599504,0.0000000
PHF20,0.000000e+00,0.0000000,0.1599972,1.319969e+00,0.0000000,1.872088e+01,1.149971,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.000000,1.250039,0.000000e+00,0.000000e+00,3.700007e-01,1.4500190,0.000000,46.5803070,0.0000000
HNF1A-AS1,2.409973e+00,2.8599942,0.9600167,2.349991e+00,1.2000098,6.300030e-01,13.370685,1.5899955,0.8100261,7.4800580,⋯,1.2900354,1.729976,0.000000,0.000000e+00,4.270046e+00,2.569959e+00,3.6600240,0.550000,1.4799789,0.0000000
CST7,0.000000e+00,0.0000000,0.0000000,0.000000e+00,0.0000000,0.000000e+00,0.000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0000000,0.000000,0.0000000,0.0000000
CD38,0.000000e+00,0.0000000,0.0000000,0.000000e+00,84.5121600,0.000000e+00,35.381367,0.0000000,0.0000000,0.0000000,⋯,31.1587730,67.028910,144.068500,0.000000e+00,4.531995e+01,0.000000e+00,0.0000000,0.000000,13.1594340,0.0000000
GRB2,0.000000e+00,412.3889000,1.8999870,0.000000e+00,87.0682800,3.657966e+01,0.000000,0.0000000,33.4693950,0.0000000,⋯,53.8594440,89.686630,40.308426,0.000000e+00,3.813879e+01,0.000000e+00,85.6191800,62.899048,78.4829800,102.9333700
COL18A1,1.595337e+02,0.0000000,0.0000000,1.008625e+02,0.0000000,0.000000e+00,18.800623,124.4036100,0.0000000,0.0000000,⋯,0.0000000,0.000000,0.000000,0.000000e+00,4.499971e+00,1.445959e+01,0.0000000,0.000000,0.0000000,7.4399996
GSTP1,5.519617e+02,576.7605600,2103.9539000,3.411219e+02,2089.7880000,2.627999e+03,8041.423300,424.7156000,2881.9524000,1846.8326000,⋯,5501.8774000,4748.292000,3577.661600,1.303616e+03,2.168513e+03,5.118524e+03,3115.4065000,2380.055000,5030.4194000,3717.5916000
RBM6,1.711484e+02,3.1700425,27.4690650,0.000000e+00,65.3496100,6.458058e+01,0.000000,141.3062900,21.5797560,156.3211800,⋯,15.6205240,98.040474,281.212430,0.000000e+00,1.162140e+02,2.030028e+00,148.3011600,0.000000,58.5900840,23.3410600


In [50]:
df_bulk

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,⋯,LINC00550,LINC01394,LINC00602,MIR1291,LINC00891,PAUPAR,GIMAP1-GIMAP5,LINC02246,HERC2P7,LINC01144
TCGA-BB-4224-01A,2237,2,1606,1063,1208,256,177,2371,2741,2515,⋯,6,7,0,2,0,6,0,16,0,144
TCGA-H7-7774-01A,2740,0,1691,803,317,330,5472,2316,8370,816,⋯,0,1,0,0,2,0,0,6,0,23
TCGA-CV-6943-01A,2686,0,1649,917,402,1535,15358,3490,4671,2080,⋯,0,4,0,2,4,1,0,38,0,18
TCGA-CN-5374-01A,2086,1,2333,1288,1105,639,5565,2065,2902,1777,⋯,0,0,1,0,3,0,0,4,0,31
TCGA-CQ-6227-01A,10167,9,3021,537,459,362,6963,4883,3140,2448,⋯,1,3,0,0,2,0,0,15,0,24
TCGA-CV-6959-01A,1154,6,2766,527,747,906,12856,6120,7448,3527,⋯,0,1,0,1,3,1,0,19,1,15
TCGA-F7-A61V-01A,1978,1,1762,482,331,147,793,2364,2827,1159,⋯,0,0,0,1,1,0,0,15,0,20
TCGA-CV-7413-01A,1930,1,1668,671,476,477,3669,3366,2324,1459,⋯,0,3,0,0,1,0,0,19,1,23
TCGA-CV-7247-01A,1066,1,1760,600,874,204,2791,1837,9779,1916,⋯,0,0,0,0,1,0,0,17,1,43
TCGA-CR-5249-01A,2538,0,1268,1066,950,924,5289,2584,2971,2572,⋯,0,0,0,4,6,0,0,17,1,27


In [42]:
if (path_marker!='noFS'){
    df_Puram_filtered <- subset(df_Puram, row.names(df_Puram) %in% marker_genes)
} else {
    df_Puram_filtered <- df_Puram
}

MuSiC_res <- run_MuSiC(df_Puram_filtered,df_bulk,label)
write.csv(MuSiC_res, paste0(path_out,"_celltypefrac_MuSiC_",FS_setup,".csv"))

Creating Relative Abundance Matrix...

Creating Variance Matrix...

Creating Library Size Matrix...



ERROR: Error in music_prop(bulk.eset = bulk_raw.eset, sc.eset = Puram.eset, clusters = "cell_category", : Too few common genes!
